In [1]:
# All initializations, to be run every time the notebook is opened
# Imports
from sklearn.ensemble import RandomForestClassifier as rfc
import json, pickle
import sys

In [2]:
history = json.load(open("history.json", "r"))[::-1]
# Keys are typedCount, title, url, lastVisitTime, visitCount, id
# Import for sentiment analysis
# Building a dataset
titles = {}
for i in range(len(history)):
    titles[i] = history[i]['title']
print len(titles)

6343


In [5]:
import lda
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
tf = CountVectorizer(stop_words='english')
tfs1 = tf.fit_transform(titles.values())

model = lda.LDA(n_topics=20, n_iter=1500, random_state=1)

model.fit_transform(tfs1)

topic_word = model.topic_word_

vocab = tf.get_feature_names()

n = 5 # 5 words for each topic
topic_word = model.topic_word_ 
topics = []
for i, topic_dist in enumerate(topic_word):
    topics.append(np.array(vocab)[np.argsort(topic_dist)][:-(n+1):-1])

print topics

INFO:lda:n_documents: 6343
INFO:lda:vocab_size: 4448
INFO:lda:n_words: 25007
INFO:lda:n_topics: 20
INFO:lda:n_iter: 1500
INFO:lda:<0> log likelihood: -302829
INFO:lda:<10> log likelihood: -186922
INFO:lda:<20> log likelihood: -177695
INFO:lda:<30> log likelihood: -174380
INFO:lda:<40> log likelihood: -172768
INFO:lda:<50> log likelihood: -171892
INFO:lda:<60> log likelihood: -171333
INFO:lda:<70> log likelihood: -170855
INFO:lda:<80> log likelihood: -170710
INFO:lda:<90> log likelihood: -170555
INFO:lda:<100> log likelihood: -170441
INFO:lda:<110> log likelihood: -170370
INFO:lda:<120> log likelihood: -170251
INFO:lda:<130> log likelihood: -169927
INFO:lda:<140> log likelihood: -169640
INFO:lda:<150> log likelihood: -169507
INFO:lda:<160> log likelihood: -169447
INFO:lda:<170> log likelihood: -169536
INFO:lda:<180> log likelihood: -169785
INFO:lda:<190> log likelihood: -169535
INFO:lda:<200> log likelihood: -169484
INFO:lda:<210> log likelihood: -169318
INFO:lda:<220> log likelihood: -

[array([u'watch', u'free', u'online', u'series', u'123movies'], 
      dtype='<U144'), array([u'jobs', u'com', u'search', u'linkedin', u'25'], 
      dtype='<U144'), array([u'google', u'search', u'careers', u'results', u'api'], 
      dtype='<U144'), array([u'amazon', u'com', u'ipad', u'apple', u'windows'], 
      dtype='<U144'), array([u'manager', u'product', u'associate', u'job', u'linkedin'], 
      dtype='<U144'), array([u'cmsc', u'671', u'grade', u'principles', u'assignment'], 
      dtype='<U144'), array([u'gmail', u'tejas', u'sathe', u'com', u'jet'], 
      dtype='<U144'), array([u'web', u'chrome', u'html', u'authentication', u'security'], 
      dtype='<U144'), array([u'engineer', u'software', u'new', u'job', u'careers'], 
      dtype='<U144'), array([u'google', u'image', u'com', u'jpg', u'2016'], 
      dtype='<U144'), array([u'django', u'admin', u'site', u'change', u'youtube'], 
      dtype='<U144'), array([u'careers', u'slack', u'center', u'sign', u'account'], 
      dtype='

In [6]:
# Classify topics into social, educational, entertainment, finance, other
# Social: pleasure +2 activity -1
# Educational: pleasure -1 activity +2
# Entertainment: pleasure +1 activity +1
# Shopping: pleasure +2 activity +2

categories = {"social": ['grade', 'teja4', 'linkedin', 'sign', 'tejas', 'edu', 'engineer', 'thank',
                         'jet', 'mail', 'gmail', 'careers', 'jobs', '671', 'job', 'sathe', 'new',
                         'account', 'cmsc', 'center', 'search', 'slack', 'facebook', 'umbc', 'com',
                         'software'], # category 1
              "educational": ['wiki', 'google', 'series', 'image', 'jpg', 'vs', 'tutorial', 'log', 'hot', 'online',
                              'web', 'chrome', 'html', 'authentication', 'security',
                              'stack', u'overflow', u'python', u'string', u'java',
                              'home', 'time', 'quora', 'page', 'memory',
                              'matlab', 'login', 'bitbucket', 'account', 'overflow',
                              'stack', 'bitbucket', 'time', 'login',
                              'security', 'page'], # category 2
              "entertainment": ['image', 'jpg', 'vs', 'online',
                                'new', 'watch', '123movies', 'vegas', 'free', 'fallout', 'naruto', 'card', 'magic',
                                'forum', 'wikia', 'youtube', 'gamefaqs', 'com'], # category 3
              "shopping": ['amazon', 'com', 'ipad', 'apple', 'windows']}

In [11]:
cats = categories.keys()
pleasure = 0
activity = 0
from random import randint
for item in history:
    title = item['title'].split()
    best_cat = 2
    best_count = 0
    for i in range(len(cats)):
        length = len(set(title).intersection(categories[cats[i]]))
        if length > best_count:
            best_cat = i+1
            best_count = length
    item["category"] = best_cat

In [12]:
import json
json.dump(history, open("dataset.json", "w"))

In [1]:
from json import load
history = load(open("dataset.json", "r"))
X, Y = [], []
for item in history:
    X.append(item['lastVisitTime'])
    Y.append(item['category'])

In [2]:
from sklearn.neighbors import KNeighborsClassifier
from pandas import DataFrame
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(DataFrame(data=X), Y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=3, p=2,
           weights='uniform')

In [6]:
from pickle import dump
dump(knn, open("knn.pickle", "wb"))